# Pencil Sketch GUI

In [1]:
import numpy as np
import cv2

In [2]:
# load the image
img = cv2.imread('./images/friends.jpg')

cv2.imshow('original',img)
cv2.waitKey()
cv2.destroyAllWindows()

![image](images/pencil_sketch_flow.PNG)

In [3]:
# step-1 convert image into grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [4]:
cv2.imshow('BGR',img)
cv2.imshow('Gray',gray)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# step-2: apply gaussian blur to the image
blur = cv2.GaussianBlur(gray,(5,5),0)

In [6]:
cv2.imshow('BGR',img)
cv2.imshow('Gray',gray)
cv2.imshow('Blur',blur)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# step-3: division image
division_img = cv2.divide(gray,blur,scale=256) # (Numerator, Denominator, scale)

In [8]:
cv2.imshow('BGR',img)
cv2.imshow('Gray',gray)
cv2.imshow('Blur',blur)
cv2.imshow('Division',division_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# step-4: adjusting gamma factor
gamma = 0.1
if gamma == 0:
    gamma = 0.01
invgamma = 1/gamma

In [10]:
# Creating Look-Up Table
lut = np.array([((i/255)**invgamma)*255 for i in range(0,256)])

# Applying Look-Up Table to division image
pencil_sketch_img = cv2.LUT(division_img.astype('uint8'),lut.astype('uint8'))

In [11]:
cv2.imshow('BGR',img)
cv2.imshow('Gray',gray)
cv2.imshow('Blur',blur)
cv2.imshow('Division',division_img)
cv2.imshow('Pencil Sketch',pencil_sketch_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
def pencil_sketch(img,ksize,gamma):
    # step-1 convert image into grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # step-2: apply gaussian blur to the image
    blur = cv2.GaussianBlur(gray,(ksize,ksize),0)
    # step-3: division image
    division_img = cv2.divide(gray,blur,scale=256)
    # step-4: adjusting gamma factor
    if gamma == 0:
        gamma = 0.01
        
    elif gamma < 0:
        raise Exception('Cannot Be Negative','Gamma value cannot be negative number (range = 0-1)')
        
    elif gamma > 1:
        raise Exception('Cannot be greater than 1','Gamma value cannot greater than 1 (range = 0-1)')

    invgamma = 1/gamma
    lut = np.array([((i/255)**invgamma)*255 for i in range(0,256)])
    pencil_sketch_img = cv2.LUT(division_img.astype('uint8'),
                                lut.astype('uint8'))
    
    return pencil_sketch_img

In [13]:
pencil_img = pencil_sketch(img,7,0.3)

In [14]:
cv2.imshow('orignial',img)
cv2.imshow('Pencil Sketch',pencil_img)

cv2.waitKey()
cv2.destroyAllWindows()

## GUI for Pencil Sketch

In [15]:
image = cv2.imread('./images/Avengers-Endgame.png')

# create control pannel
cv2.namedWindow('Control Pannel')
cv2.createTrackbar('ksize','Control Pannel',1,100,(lambda x:None))
cv2.createTrackbar('gamma','Control Pannel',1,100,(lambda x:None))

while True:
    
    # take k-size
    k = cv2.getTrackbarPos('ksize','Control Pannel')
    # k size should be an odd number
    k = 2*k + 1
    # calibrate gamma to 0 - 1 scale
    g = cv2.getTrackbarPos('gamma','Control Pannel')
    g = g/100
    pencil_img = pencil_sketch(image,k,g)
    
    
    cv2.imshow('original',image)
    cv2.imshow('Pencil Sketch',pencil_img)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()

## Real Time Pencil Sketch GUI For Video

In [16]:
cap0 = cv2.VideoCapture(0)

# create control pannel
cv2.namedWindow('Control Pannel')
cv2.createTrackbar('ksize','Control Pannel',1,100,(lambda x:None))
cv2.createTrackbar('gamma','Control Pannel',1,100,(lambda x:None))

while True:
    ret, frame = cap0.read()
    # take k-size
    k = cv2.getTrackbarPos('ksize','Control Pannel')
    
    # k size should be an odd number
    k = 2*k + 1
    
    # calibrate gamma to 0 - 1 scale
    g = cv2.getTrackbarPos('gamma','Control Pannel')
    g = g/100
    
    pencil_frame = pencil_sketch(frame,k,g)

    if ret == False:
        break
    if cv2.waitKey(1) == 27:
        break
    cv2.imshow('LIVE', frame)
    cv2.imshow('LIVE Sketch', pencil_frame)

cap0.release()
cv2.destroyAllWindows()